# **Connect drive with file**

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import random
from numpy.random import choice

# **Read files**

In [ ]:
# Read the excel files uploaded on da Google drive
df_theory = pd.read_excel("/content/drive/My Drive/courses.xlsx")
df_lab = pd.read_excel("/content/drive/My Drive/lab_courses.xlsx")
timetable_theory = pd.read_excel("/content/drive/My Drive/timetable.xlsx")
timetable_lab = pd.read_excel("/content/drive/My Drive/timetable_lab.xlsx")

In [ ]:
# Extract room names and timeslot names for theory and lab
rooms_theory = timetable_theory['Room'].values.tolist()[:30]
timeslots_theory = timetable_theory.columns.values.tolist()[1:]
rooms_lab = timetable_lab['Lab'].values.tolist()[:16]
timeslots_lab = timetable_lab.columns.values.tolist()[1:]

In [ ]:
# concat theory and lab courses and timetables
df_combined = pd.concat([df_theory, df_lab], ignore_index=True)
timetable_combined = pd.concat([timetable_theory, timetable_lab], axis=1)

# Separate theory and lab chromosomes(this is to ensure that the time for both lab and thoery does not overlap with each other)
df_theory['Type'] = 'Theory'
df_lab['Type'] = 'Lab'
df_combined = pd.concat([df_theory, df_lab], ignore_index=True)

In [ ]:
# Create chromosomes using binary encoding
courses = df_combined['Course'].unique()
sections = df_combined['Section'].unique()
instructors = df_combined['Course Instructor'].unique()

course_bits = len(format(len(courses), 'b'))
section_bits = len(format(len(sections), 'b'))
instructor_bits = len(format(len(instructors), 'b'))

# Create a mapping for courses, sections, and instructors to binary strings
course_mapping = {course: format(index, f'0{course_bits}b') for index, course in enumerate(courses)}
section_mapping = {section: format(index, f'0{section_bits}b') for index, section in enumerate(sections)}
instructor_mapping = {instructor: format(index, f'0{instructor_bits}b') for index, instructor in enumerate(instructors)}

In [ ]:
# Create reverse mappings for decoding(this will be used by the fitness function to check if after crossover and mutation, a new chromosome should be amongst the one that are already present in the list)
reverse_course_mapping = {v: k for k, v in course_mapping.items()}
reverse_section_mapping = {v: k for k, v in section_mapping.items()}
reverse_instructor_mapping = {v: k for k, v in instructor_mapping.items()}


In [ ]:
# chromosomes with binary encoding for theory
theory_chromosomes = np.array([
    [course_mapping[row['Course']], section_mapping[row['Section']], instructor_mapping[row['Course Instructor']]]
    for _, row in df_combined[df_combined['Type'] == 'Theory'].iterrows()
])

In [ ]:
# chromosomes with binary encoding for lab
lab_chromosomes = np.array([
    [course_mapping[row['Course']], section_mapping[row['Section']], instructor_mapping[row['Course Instructor']]]
    for _, row in df_combined[df_combined['Type'] == 'Lab'].iterrows()
])

In [ ]:
# Store binary and string values in a dictionary for decoding
binary_string_mapping = {
    'course': {v: k for k, v in course_mapping.items()},
    'section': {v: k for k, v in section_mapping.items()},
    'instructor': {v: k for k, v in instructor_mapping.items()}
}

In [ ]:
# Dictionary to keep track of courses assigned to each teacher
teacher_course_count = {teacher: 0 for teacher in instructor_mapping.values()}

# Dictionary to keep track of courses assigned to each section
section_course_count = {section: 0 for section in section_mapping.values()}

In [ ]:
# Fitness function
# There are normal scores and heavy penality. heavy penality would be for breaking hard constraints (such as teacher having clash or a section having clash)
def fitness(timetable):
    score = 0

    # If a column doesn't have a section name clash, +40 points
    for x in timetable.T[1]:
        if len(np.unique(x)) == len(x):
            score += 40

    # If a column doesn't have a teacher name clash, +40 points
    for x in timetable.T[2]:
        if len(np.unique(x)) == len(x):
            score += 40

    # Check for clashes of classes at the same time
    if timetable.shape[1] == len(timeslots_theory):
        timeslots = timeslots_theory
    else:
        timeslots = timeslots_lab

    for timeslot_index in range(len(timeslots)):
        section_teacher_pairs = timetable[:, timeslot_index, 1:3]  # Extract section and teacher for the current timeslot
        unique_pairs = np.unique(section_teacher_pairs, axis=0)  # Get unique pairs of section and teacher
        if len(unique_pairs) != len(section_teacher_pairs):  # If there are duplicate pairs, penalize
            score -= 100  # Penalize

    # Check if a professor is assigned more than 3 courses
    for teacher, count in teacher_course_count.items():
        if count > 3:
            score -= (count - 3) * 100  # Penalize

    # Check if no section has more than 5 courses
    for section, count in section_course_count.items():
        if count > 5:
            score -= (count - 5) * 100  # Penalize

    return score

In [ ]:
# Mutation function
#Mutation in this timetable context is like randomly changing some course sections and their assigned instructors,
#hoping to improve the overall timetable. We randomly decide which sections to change, and then assign them different
#instructors, but ensuring they're not already assigned to that section.
def mutation(child, probability):
    if random.random() < probability:
        for index in range(len(child)):
            for i in range(len(child[index])):
                if len(np.unique(child[:, i, 1])) == 1:
                    new_chromosome = random.choice(chromosomes)
                    while not np.array_equal(new_chromosome, child[index, i]):
                        new_chromosome = random.choice(chromosomes)
                    child[index, i] = new_chromosome
    return child

In [ ]:
# Roulette wheel selection based on index and their respective scores
def selection(index, scores):
    # Convert scores to non-negative values
    min_score = min(scores)
    non_negative_scores = scores - min_score + 1  # Add 1 to ensure non-negative scores
    distribution = non_negative_scores / non_negative_scores.sum()
    return choice(index, 2, p=distribution, replace=False)

In [ ]:
# Single point crossover to create new children
def crossover(parent1, parent2):
    if random.random() < 0.5:
        index = random.randint(1, len(parent1) - 2)
        child1 = np.concatenate((parent1[:index], parent2[index:]))
        child2 = np.concatenate((parent2[:index], parent1[index:]))
    else:
        index = random.randint(1, len(timeslots_theory) - 2)
        child1 = np.concatenate((parent1[:, :index], parent2[:, index:]), axis=1)
        child2 = np.concatenate((parent1[:, :index], parent2[:, index:]), axis=1)

    # Ensure new chromosomes are among the list of dictionaries)
    for i in range(len(child1)):
        for j in range(len(child1[i])):
            chromosome_index = np.where((chromosomes == child1[i, j]).all(axis=1))[0][0]
            if not np.array_equal(child1[i, j], chromosomes[chromosome_index]):
                child1[i, j] = random.choice(chromosomes)
            chromosome_index = np.where((chromosomes == child2[i, j]).all(axis=1))[0][0]
            if not np.array_equal(child2[i, j], chromosomes[chromosome_index]):
                child2[i, j] = random.choice(chromosomes)

    return [child1, child2]

In [ ]:
# Generate initial population
populationSize = min(len(rooms_theory), len(rooms_lab))
generations = 1000
mutationProb = 0.2

# Initialize empty timetables for each day of the week
timetables = {day: np.array([[[] for _ in range(len(timeslots_theory) + len(timeslots_lab))] for _ in range(populationSize)]) for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']}


In [ ]:
# Evolutionary loop for each day
for day in timetables.keys():
    # Separate populations for theory and lab
    theory_population = [np.array([[random.choice(theory_chromosomes) for _ in timeslots_theory] for _ in range(populationSize)]) for _ in range(populationSize)]
    lab_population = [np.array([[random.choice(lab_chromosomes) for _ in timeslots_lab] for _ in range(populationSize)]) for _ in range(populationSize)]

    for gen in range(generations):
        # Fitness evaluation for theory and lab separately so that the course instructors and sections dont overlap with each other
        theory_scores = np.array([fitness(p) for p in theory_population])
        lab_scores = np.array([fitness(p) for p in lab_population])

        newTheoryPopulation = []
        newLabPopulation = []

        # Crossover and mutation for theory population
        for i in range(populationSize // 2):
            parents = selection(np.arange(0, populationSize), theory_scores)
            children = crossover(theory_population[parents[0]], theory_population[parents[1]])
            children[0] = mutation(children[0], mutationProb)
            children[1] = mutation(children[1], mutationProb)
            newTheoryPopulation.extend(children)
        # Crossover and mutation for lab population
        for i in range(populationSize // 2):
            parents = selection(np.arange(0, populationSize), lab_scores)
            children = crossover(lab_population[parents[0]], lab_population[parents[1]])
            children[0] = mutation(children[0], mutationProb)
            children[1] = mutation(children[1], mutationProb)
            newLabPopulation.extend(children)

        theory_population = sorted(theory_population + newTheoryPopulation, key=fitness, reverse=True)[:populationSize]
        lab_population = sorted(lab_population + newLabPopulation, key=fitness, reverse=True)[:populationSize]

        if gen % 100 == 0:
            print(f"Day: {day}, Generation - {gen + 1}")
            print(f"Best Theory Score: {fitness(theory_population[0])}")
            print(f"Best Lab Score: {fitness(lab_population[0])}")

            # Check if the best timetables fulfill all constraints
            if fitness(theory_population[0]) == 0 and fitness(lab_population[0]) == 0:
                print("Timetables found meeting all constraints!")
                break

    # Combine best theory and lab timetables for the day
    combined_timetable = np.concatenate((theory_population[0], lab_population[0]), axis=1)
    timetables[day] = combined_timetable


Day: Monday, Generation - 1
Best Theory Score: 360
Best Lab Score: 120
Day: Monday, Generation - 101
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 201
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 301
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 401
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 501
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 601
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 701
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 801
Best Theory Score: 440
Best Lab Score: 200
Day: Monday, Generation - 901
Best Theory Score: 440
Best Lab Score: 200
Day: Tuesday, Generation - 1
Best Theory Score: 240
Best Lab Score: -20
Day: Tuesday, Generation - 101
Best Theory Score: 480
Best Lab Score: 120
Day: Tuesday, Generation - 201
Best Theory Score: 480
Best Lab Score: 120
Day: Tuesday, Generation - 301
Best Theory Score: 48

# **Display the timetable after successful mutation**

In [ ]:
# Display the timetables for theory and lab courses for each day
for day in timetables.keys():
    print(f"\n{day} Timetable for Theory Courses:")
    decoded_theory_timetable = timetables[day][:, :len(timeslots_theory)]
    theory_timetable_with_sections = np.array([
        [
            f"{binary_string_mapping['course'][cell[0]]} ({binary_string_mapping['section'][cell[1]]}) ({binary_string_mapping['instructor'][cell[2]]})"
            if len(cell) == 3 else ""
            for cell in row
        ]
        for row in decoded_theory_timetable[:len(rooms_theory)]
    ])

    finalTheoryTimeTable = pd.DataFrame(theory_timetable_with_sections, columns=timeslots_theory, index=rooms_theory[:len(theory_timetable_with_sections)])
    display(finalTheoryTimeTable)

    print(f"\n{day} Timetable for Lab Courses:")
    decoded_lab_timetable = timetables[day][:, len(timeslots_theory):]
    lab_timetable_with_sections = np.array([
        [
            f"{binary_string_mapping['course'][cell[0]]} ({binary_string_mapping['section'][cell[1]]}) ({binary_string_mapping['instructor'][cell[2]]})"
            if len(cell) == 3 else ""
            for cell in row
        ]
        for row in decoded_lab_timetable
    ])

    finalLabTimeTable = pd.DataFrame(lab_timetable_with_sections, columns=timeslots_lab, index=rooms_lab)
    display(finalLabTimeTable)



Monday Timetable for Theory Courses:


,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
C-301,Introduction to Software Engineering (SE) (SE-...,Artificial Intelligence (CS) (CS-6B) (Mr. Saad...,Probability and Statistics (CS) (CS-4E) (Mooma...,Probability and Statistics (CS) (CS-4H) (Sana ...,Data Mining (CS-8A) (Dr. Ahmad Raza Shahid),Programming Fundamentals (All Programs) (2G) (...
C-302,Communication and Presentation Skills (SE) (S...,Islamic Studies/Ethics (CY) (CY-2B) (Sobia She...,MS Thesis - 2 (MCS-A) (MSRC2),Fundamentals of Management (CS) (CS-4B) (Ayesh...,Vulnerability Assessment & Reverse Engineering...,Pakistan Studies (CS) (CS-2A) (Farooq Dar)
C-303,Marketing Management (CS) (CS-4D) (Tanzeela Akif),Calculus and Analytical Geometry (All Programs...,Advanced Software Requirement Engineering (MSE...,Object Oriented Programming (CY) (CY-2A) (Mr. ...,Digital Logic Design (CS) (CS-2A) (Ms. Nirmal ...,Artificial Neural Networks (AI) (AI-6K) (Mr. H...
C-304,Advanced Computer Vision (PCS-2A) (Dr. Ahmad R...,Object Oriented Programming (CS) (CS-2C) (Mr. ...,Database Systems (CS) (CS-4B/Z) (Dr. Shujaat H...,Differential Equations (CS) (CS-2G) (Sumaira A...,Object Oriented Programming (CS) (CS-2G) (Mr. ...,Machine Learning (BS-AI) (AI-A) (Dr. Uzair Iqbal)
C-305,Programming Fundamentals (All Programs) (2H) (...,Operating Systems (CS) (CS-4C/D) (Mr. Shoaib M...,Fundamentals of Software Project Management (C...,Introduction to Software Engineering (SE) (SE-...,Communication and Presentation Skills (CY) (CY...,Information Security (SE) (SE-6S) (Ms. Hina Bi...
C-307,Differential Equations (CY) (CY-2D) (Javaria A...,Discrete Structures (DS) (DS-2D) (Dr. Abid Rauf),Differential Equations (DS) (DS-2A) (Sumaira A...,Digital Logic Design (CS) (CS-2K) (Dr. Mehwish...,Artificial Intelligence (CS) (CS-6C) (Mr. Saad...,Pakistan Studies (CS) (CS-2F) (Muhammad Ajmal)
C-308,Differential Equations (DS) (DS-2A) (Sumaira A...,Web Engineering (SE) (SE-6Q) (Mr. Zaheer Sani),Final Year Project-I (CY) (CY-T) (FYP Committee),Probability and Statistics (SE) (SE-4Q) (Khali...,Applied Evolutionary Computing (AI) (AI-8K) (D...,Probability and Statistics (CS) (CS-4E) (Mooma...
C-309,Probability and Statistics (SE) (SE-4Q) (Khali...,Design and Analysis of Algorithms (CS) (CS-4B/...,Theory of Automata (CS-A) (Ms. Tajwar Mehmood),Computer Organization and Assembly Language (A...,Probability and Statistics (CS) (CS-4C) (Waqas...,Communication and Presentation Skills (DS) (DS...
C-310,Natural Language Processing (MDS-A) (Dr. Mehr...,Web Programming (CS-B) (Ms. Tajwar Mehmood),Introduction to Software Engineering (SE) (SE-...,Object Oriented Programming (CS) (CS-2C) (Mr. ...,Pakistan Studies (CS) (CS-2B) (Farooq Dar),Final Year Project-I (CY) (CY-T) (FYP Committee)
C-311,Pakistan Studies (CS) (CS-2G) (Gul-e-Aisha),Discrete Structures (CY) (CY-2A) (Ms. Sobia Ra...,Islamic Studies/Ethics (DS) (DS-2C) (Abdul Sa...,Object Oriented Programming (SE) (SE-2G) (Ms. ...,Applied Artificial Intelligence (SE Elective -...,Parallel and Distributed Computing (AI) (AI-6J...



Monday Timetable for Lab Courses:


,08:30-11:20,11:30-2:20,2:30-5:15
C-Margala 1,Database Systems (CS) (CS-4G) (Ms. Amina Siddi...,Operating Systems (CS) (CS-4G) (Ms. Sana Razzaq),Object Oriented Programming (CS) (CS-2E) (Ms. ...
C-Margala 3,Database Systems (CS) (CS-4A/Y) (Ms. Urooj Ghani),Programming Fundamentals (All Programs) (2D) (...,Database Systems (CS) (CS-4A/Y) (Ms. Urooj Ghani)
C-Margala 4,Object Oriented Programming (DS) (DS-2D) (Ms. ...,Computer Networks (AI) (AI-6J) (Ms. Maham Haider),Object Oriented Programming (SE) (SE-2C) (Mr. ...
C-Rawal 1,Artificial Intelligence (CS) (CS-6C) (Mr. Muha...,Artificial Intelligence (AI) (AI-4J) (Ms. Bush...,Data Structures (All Programs) (4E) (Ms. Hira ...
Rawal 3 (B-232),Vulnerability Assessment & Reverse Engineering...,Artificial Intelligence (CY) (CY-6T) (Mr. Muha...,Data Mining (DS) (DS-6N) (Ms. Bushra Amjad)
C-Rawal 4,Data Structures (All Programs) (4F) (Mr. Shoai...,Data Structures (All Programs) (4G) (Ms. Hira ...,Computer Networks (SE) (SE-6P) (Mr. Abdullah A...
A-Karakoram 1,Database Systems (DS) (DS-4N) (Mr. Kumail Naqvi),Object Oriented Programming (CY) (CY-2D) (Ms. ...,Object Oriented Programming (CS) (CS-2A) (Ms. ...
A-Karakoram 2,Artificial Intelligence (CS) (CS-6A) (Mr. Muha...,Object Oriented Programming (CY) (CY-2A) (Ms. ...,Artificial Intelligence (CS) (CS-6E) (Mr. Muha...
A-Karakoram 3,Computer Networks (SE) (SE-6Q) (Mr. Abdullah A...,Object Oriented Programming (AI) (AI-2B) (Ms. ...,Object Oriented Programming (SE) (SE-2B) (Ms. ...
A-Mehran 1,Database Systems (SE) (SE-4P) (Mr. Obaid Ullah...,Computer Networks (SE) (SE-6Q) (Mr. Abdullah A...,Data Structures (All Programs) (4C) (Ms. Sara ...



Tuesday Timetable for Theory Courses:


...

,08:30-11:20,11:30-2:20,2:30-5:15
C-Margala 1,Object Oriented Programming (AI) (AI-2C) (Ms. ...,Artificial Intelligence (CS) (CS-6B) (Mr. Muha...,Software Design and Architecture (SE) (SE-4P) ...
C-Margala 3,Object Oriented Programming (CS) (CS-2E) (Ms. ...,Object Oriented Programming (CS) (CS-2B) (Ms. ...,Operating Systems (CS) (CS-4A/Y) (Ms. Javeria ...
C-Margala 4,Artificial Intelligence (CS) (CS-6F) (Mr. Muha...,Programming Fundamentals (All Programs) (2C) (...,Object Oriented Programming (DS) (DS-2A) (Ms. ...
C-Rawal 1,Programming Fundamentals (All Programs) (2H) (...,Programming Fundamentals (All Programs) (2B) (...,Object Oriented Programming (SE) (SE-2C) (Mr. ...
Rawal 3 (B-232),Database Systems (DS) (DS-4N) (Mr. Kumail Naqvi),Object Oriented Programming (CS) (CS-2D) (Ms. ...,Object Oriented Programming (AI) (AI-2B) (Ms. ...
C-Rawal 4,Artificial Intelligence (CS) (CS-6B) (Mr. Muha...,Operating Systems (CS) (CS-4C/D) (Ms. Javeria ...,Artificial Intelligence (DS) (DS-6N) (Mr. Umai...
A-Karakoram 1,Object Oriented Programming (CY) (CY-2C) (Ms. ...,Data Structures (All Programs) (4E) (Ms. Hira ...,Programming Fundamentals (All Programs) (2D) (...
A-Karakoram 2,Programming Fundamentals (All Programs) (2C) (...,Object Oriented Programming (SE) (SE-2A) (Mr. ...,Object Oriented Programming (SE) (SE-2F) (Ms. ...
A-Karakoram 3,Object Oriented Programming (SE) (SE-2E) (Ms. ...,Artificial Intelligence (CS) (CS-6C) (Mr. Muha...,Object Oriented Programming (CS) (CS-2A) (Ms. ...
A-Mehran 1,Object Oriented Programming (CY) (CY-2D) (Ms. ...,Fundamentals of Software Engineering (DS) (DS-...,Object Oriented Programming (CS) (CS-Z) (Ms. M...



Friday Timetable for Theory Courses:


,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
C-301,Fundamentals of Management (CS) (CS-4A) (Ayesh...,Fundamentals of Big Data Analytics (DS) (DS-4M...,Parallel and Distributed Computing (CS) (CS-6F...,Database Systems (SE) (SE-4P) (Ms. Hina Binte ...,Software Engineering (CS) (CS-6B) (Ms. Sidra K...,Islamic Studies/Ethics (SE) (SE-2E) (Asad Ullah)
C-302,Database Systems (CS) (CS-4C/D) (Ms. Noor ul Ain),Research Methodology (MCS-A) (Dr. Hammad Majeed),Object Oriented Programming (SE) (SE-2B) (Ms. ...,Search-based Software Engineering (MCS-A) (Dr....,Communication and Presentation Skills (SE) (S...,Applied Physics (CS-B) (Aisha Ijaz)
C-303,MS Thesis - 1 (MCNS-A) (MSRC),Data Structures (All Programs) (4F) (Mr. Rohai...,Technical and Business Writing (All Programs) ...,Distributed Data Engineering Systems (MCS-A) (...,MS Thesis - 1 (MAI-A) (MSRC),Machine Learning for Data Science (MDS-A) (Dr....
C-304,Ethical Hacking Concepts and Practices (CY) (C...,Introduction to Software Engineering (SE) (SE-...,Pakistan Studies (CS) (CS-2A) (Farooq Dar),Final Year Project-II (DS) (DS-N) (FYP Committ...,Discrete Structures (CY) (CY-2A) (Ms. Sobia Ra...,Object Oriented Programming (SE) (SE-2E) (Mr. ...
C-305,Final Year Project-II (DS) (DS-N) (FYP Committ...,Vulnerability Assessment & Reverse Engineering...,Applied Image Processing (PCS-3A) (Dr. Akhtar ...,Process Mining and Simulation (SE Supporting-I...,Operating Systems (CS) (CS-4E/V) (Mr. Shoaib M...,Professional Practices in IT (AI) (AI-8K) (Dr....
C-307,Differential Equations (CS) (CS-2J) (Hamda Khan),Distributed Data Engineering (CS-8A) (Dr. Adna...,Web Engineering (SE) (SE-6Q) (Mr. Zaheer Sani),Digital Image Processing (AI Elective-1) (AI-6...,Digital Logic Design (CS) (CS-2G) (Dr. Niaz Ah...,Probability and Statistics (CS) (CS-4A) (Khali...
C-308,Pakistan Studies (CS) (CS-2E) (Maimoona Rasool),Communication and Presentation Skills (AI) (AI...,Process Mining and Simulation (SE Supporting-I...,Object Oriented Programming (DS) (DS-2B) (Mr. ...,Islamic Studies/Ethics (DS) (DS-2B) (Yahya Ma...,Fundamentals of Software Engineering (DS) (DS-...
C-309,Pakistan Studies (CS) (CS-2H) (Fouzia Umer),Web Programming (CS-B) (Ms. Tajwar Mehmood),Natural Language Processing (CS-8A) (Dr. Mehre...,Advanced Security Operations & Management (MCN...,Advanced Statistics (DS) (DS-4M) (Dr. Ahmed Din),Communication and Presentation Skills (SE) (S...
C-310,MLOPs (CS-8B) (Mr. Shahbaz Hussain (VF)),Parallel and Distributed Computing (CS) (CS-6F...,Numerical Computing (CS) (CS-6B) (Muhammad Ali),Business Strategy (CS)\n(Pre-Req: Fundamentals...,Cloud Computing (CY-4T) (Dr. Mudassar Aslam),Introduction to Software Engineering (SE) (SE-...
C-311,Computer Networks (SE) (SE-6P) (Mr. Abdullah A...,Object Oriented Programming (CS) (CS-2A) (Mr. ...,Object Oriented Programming (AI) (AI-2A) (Mr. ...,Probability and Statistics (CS) (CS-4H) (Sana ...,Digital Logic Design (CS) (CS-2K) (Dr. Mehwish...,Statistical and Mathematical Methods for DS (M...



Friday Timetable for Lab Courses:


,08:30-11:20,11:30-2:20,2:30-5:15
C-Margala 1,Artificial Intelligence (CS) (CS-6D) (Mr. Muha...,Object Oriented Programming (CS) (CS-Z) (Ms. M...,Object Oriented Programming (CS) (CS-2J) (Ms. ...
C-Margala 3,Database Systems (AI) (AI-4J) (Ms. Ayesha Kamran),Object Oriented Programming (SE) (SE-2B) (Ms. ...,Software Design and Architecture (SE) (SE-4Q) ...
C-Margala 4,Data Structures (All Programs) (4G) (Ms. Hira ...,Web Programming (DS-Elective 1) (DS-6N) (Mr. S...,Programming Fundamentals (All Programs) (2D) (...
C-Rawal 1,Object Oriented Programming (SE) (SE-2E) (Ms. ...,Object Oriented Programming (CS) (CS-2K) (Ms. ...,Object Oriented Programming (CS) (CS-2K) (Ms. ...
Rawal 3 (B-232),Object Oriented Programming (CS) (CS-2F) (Ms. ...,Object Oriented Programming (CS) (CS-2F) (Ms. ...,Object Oriented Programming (CS) (CS-2G) (Ms. ...
C-Rawal 4,Object Oriented Programming (CS) (CS-2B) (Ms. ...,Database Systems (DS) (DS-4N) (Mr. Kumail Naqvi),Artificial Intelligence (CS) (CS-6A) (Mr. Muha...
A-Karakoram 1,Database Systems (CS) (CS-4C/D) (Ms. Urooj Ghani),Artificial Intelligence (CS) (CS-6D) (Mr. Muha...,Object Oriented Programming (CS) (CS-2F) (Ms. ...
A-Karakoram 2,Object Oriented Programming (CY) (CY-2B) (Ms. ...,Object Oriented Programming (CS) (CS-2H) (Ms. ...,Object Oriented Programming (CS) (CS-2C) (Ms. ...
A-Karakoram 3,Object Oriented Programming (SE) (SE-2B) (Ms. ...,Object Oriented Programming (CS) (CS-2C) (Ms. ...,Programming Fundamentals (All Programs) (2C) (...
A-Mehran 1,Data Structures (All Programs) (4B) (Ms. Huma ...,Database Systems (CS) (CS-4F/H) (Ms. Amina Sid...,Object Oriented Programming (DS) (DS-2D) (Ms. ...
